In [1]:
import glob
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import zscore
from tensorflow import keras

In [2]:
# Constants
# batch size for model
BATCH_SIZE = 500
# number of epochs
EPOCHS = 20
# proportion of validation data
VALIDATION_SPLIT = 0.2
# learning rate and epsilon for ADAM optimizer
LEARNING_RATE = 0.01
EPSILON = 1
# path where data is stored
PATH="/Users/tanvipotdar/Projects/thesis/data/INTC_2015-01-01_2015-01-31_10"
# prediction horizon
K = 50 
# threshold to decide which category midprice direction falls in (up, down, stationary)
ALPHA = 0.001

# Static objects
# instantiate one hot encoder here so that all classes will always map to the same labels
onehot_encoder = OneHotEncoder(sparse=False)

### Prepare the data

In [ ]:
def get_data(path):
    all_files = glob.glob(path + "/*orderbook_10.csv")
    all_files.sort()
    orderbooks = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=None)
        orderbooks.append(df)
    orderbook = pd.concat(orderbooks, axis=0, ignore_index=True)
    col_names = ['ask_price_', 'ask_size_', 'bid_price_', 'bid_size_']
    nums = map(str, range(1,11))
    orderbook.columns = [y + x for x in nums for y in col_names]
    return orderbook
data = get_data(path=PATH)
data.head()

In [ ]:
def normalise_data(data):
    normalised_data = data.apply(zscore)
    return normalised_data
normalised_data = normalise_data(data)
normalised_data.head()

In [ ]:
# smoothed labelling of the midprice/ K is the prediction horizon
def smooth_midprice_using_k_lookahead(normalised_data, k):
    normalised_data['midprice'] = (normalised_data.ask_price_1+normalised_data.bid_price_1)/2
    # mean of previous k mid-prices
    normalised_data['m_minus'] = normalised_data['midprice'].rolling(window=k).mean()
    # mean of next k mid-prices
    normalised_data['m_plus'] = normalised_data['midprice'][::-1].rolling(window=k).mean()[::-1]
    return normalised_data
normalised_data = smooth_midprice_using_k_lookahead(normalised_data, k=K)

In [ ]:
# label the smoothed mid-prices based on a threshold/ ALPHA is the threshold 
def create_midprice_labels(normalised_data):
    normalised_data['change'] = (normalised_data.m_plus - normalised_data.m_minus)/normalised_data.m_minus
    # assign categories up, down, stationary
    normalised_data['label'] = pd.cut(normalised_data.change, bins=[-np.inf, -ALPHA, ALPHA, np.inf], 
                                    labels=['down', 'stationary', 'up'])
    # drop all unlabelled values (will be first and last k values as they have no m_minus/m_plus value)
    normalised_data.dropna(inplace=True)
    return normalised_data
normalised_data = create_midprice_labels(normalised_data)

In [ ]:
# split into train and test data
N = len(normalised_data) - len(normalised_data)%100
def reshape_and_categorise_data(normalised_data, n):
    data = normalised_data[:n]
    cols = data.columns.to_list()[:40]
    input_data = data[cols]
    input_array = input_data.to_numpy().reshape(n//100,100,40,1)

    output_data = data.label.to_numpy()[::-100][::-1]
    integer_encoded = output_data.reshape(len(output_data), 1)
    output_array = onehot_encoder.fit_transform(integer_encoded)
    X_train, X_test, y_train, y_test = train_test_split(input_array, output_array, shuffle=False)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = reshape_and_categorise_data(normalised_data, N)
print("Training input shape:",X_train.shape)
print("Test input shape:", X_test.shape)
print("Training output shape:", y_train.shape)
print("Test input shape:", y_test.shape)

In [ ]:
# mapping of labels to one hot encoding
encodings = [[1,0,0],[0,0,1],[0,1,0]]
classes = onehot_encoder.inverse_transform(encodings)
print(zip(classes, encodings))
# Using np.argmax equates: 0-down, 1-stationary, 2-up

### Build and compile the model

In [ ]:
def create_model():
    # convolutional layers
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(1,2), input_shape=(100,40,1), strides=(1, 2)))
    model.add(keras.layers.LeakyReLU(alpha=0.01))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(4,1)))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(4,1)))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1, 2)))
    model.add(keras.layers.LeakyReLU(alpha=0.01))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(4,1)))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(4,1)))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(1,10), input_shape=(100,10,1)))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=(4,1)))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=(4,1)))
    model.add(keras.layers.MaxPooling2D(pool_size=(3,1)))
    model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

    # lstm layer
    model.add(keras.layers.LSTM(100))
    model.add(keras.layers.Dense(3,activation='softmax'))
    # compile model and summarize
    adam = keras.optimizers.Adam(lr=LEARNING_RATE, epsilon=1)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
model = create_model()

In [ ]:
def fit_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT)
    score, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
    print("Accuracy is {}%".format(accuracy*100))
    return accuracy*100, history
accuracy, history = fit_and_evaluate_model(model, X_train, X_test, y_train, y_test)

In [ ]:
def plots(history):
    plt.figure(figsize=(15,10))
    plt.subplot(2,1,1)
    plt.plot(history.history['loss'],'b--',lw=2,label='train_loss')
    plt.plot(history.history['val_loss'],'g-',lw=2,label='val_loss')
    plt.legend()
    # plt.ylim([.5,1.3])
    plt.xlabel('Number of epochs')
    plt.ylabel('Loss')
    plt.subplot(2,1,2)
    plt.plot(history.history['acc'],'b--',lw=2,label='train_acc')
    plt.plot(history.history['val_acc'],'g-',lw=2,label='val_acc')
    plt.legend()
    # plt.ylim([.2,1.0])
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')
    plt.show()
plots(history)

### Evaluate performance

In [ ]:
def get_report(model, X_test, y_test, X_train, y_train):
    target_names = ['down', 'stationary', 'up']
    y_pred = model.predict(X_test)
    y_pred_bool = np.argmax(y_pred, axis=1)
    y_test_bool = np.argmax(y_test, axis=1)
    print(classification_report(y_pred_bool, y_test_bool, target_names=target_names))
    print(confusion_matrix(y_test_bool, y_pred_bool, labels=[0,1,2]))
get_report(model, X_test, y_test, X_train, y_train)